In [ ]:
#assignment2_BigData

In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-3.0.0/spark-3.0.0-bin-hadoop3.2.tgz
!tar xf spark-3.0.0-bin-hadoop3.2.tgz

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.0-bin-hadoop3.2"

In [ ]:
!pip install -q findspark

In [ ]:
import findspark
findspark.init()
from pyspark.sql import SparkSession

spark = SparkSession.builder.master("local[*]").getOrCreate()

In [ ]:
from pyspark import SparkConf 
from pyspark.context import SparkContext 

sc = SparkContext.getOrCreate(SparkConf().setMaster("local[*]"))

In [ ]:
#Retrieve the first k records : k = 10
dataset = sc.textFile("/content/pagecounts-20160101-000000_parsed.out",minPartitions=0) 
sc.parallelize(dataset.take(10)).saveAsTextFile('one1211.tx')

In [ ]:
#maximum
def parse_line(line):
  colloms = line.split(" ")
  pagesize=float(colloms[3])
  key=0
  return (key,pagesize)
data =sc.textFile("/content/pagecounts-20160101-000000_parsed.out")
pagesize=data.map(parse_line)
maxvalue = pagesize.reduceByKey(lambda x , y: max(x,y))
maxvalue.saveAsTextFile('max.tx')
#maxTemps.collect()

In [ ]:
#minimum
def parse_line(line):
  colloms = line.split(" ")
  pagesize=float(colloms[3])
  key=0
  return (key,pagesize)
data =sc.textFile("/content/pagecounts-20160101-000000_parsed.out")
pagesize=data.map(parse_line)
minvlues = pagesize.reduceByKey(lambda x , y: min(x,y))
minvlues.saveAsTextFile('min1.tx')

In [ ]:
#average
def parse_data(line):
 colloms = line.split(" ")
 page_size=float(colloms[3])
 return (page_size)
data =sc.textFile("/content/pagecounts-20160101-000000_parsed.out")
page_size=data.map(parse_data)
listavg =[]
result_avg=page_size.mean()
listavg.append(result_avg)
final_rdd= sc.parallelize(listavg)
final_rdd.saveAsTextFile('avg1.tx')

In [ ]:
#page titles that start with the article‎“The”
import re
def parse_data(line):
  colloms = line.split(" ")
  return (colloms[0],colloms[1])
#lines.take(5)
data =sc.textFile("/content/pagecounts-20160101-000000_parsed.out")
pagetitle=data.map(parse_data)
result=pagetitle.filter(lambda x : (x[1].startswith('The')) & ('en' != x[0]))
#Newr.take(5)
pagetitlecount=result.distinct().count()
list_count=[]
list_count.insert(0,pagetitlecount) #index,value
rdd3= sc.parallelize(list_count)
rdd3.saveAsTextFile('special141.tx')

In [ ]:
#Task 4
import re
def parseLine(line):
  fields = line.split(' ')
  title = fields[1]
  return (title)
def lower_clean_str(x):
  punc= re.sub(r'[^a-zA-Z_]','', x)
  lowercased_str=punc.lower()
  return lowercased_str
lines=sc.textFile("/content/pagecounts-20160101-000000_parsed.out")
parsedLines=lines.map(parseLine).flatMap(lambda line: line.split("_"))
rdd=parsedLines.map(lower_clean_str)
result=rdd.map(lambda title : (title, 1)).reduceByKey(lambda x , y: x+y).filter(lambda x: x[1]==1).count()
#print(result)
list_count=[]
list_count.insert(0,result)
sc.parallelize(list_count).saveAsTextFile('UTerms1az.tx')

973059


In [ ]:
#Task 5
sc=SparkContext.getOrCreate();
def parseLine(line):
  fields=line.split(' ')
  title=fields[1]
  return (title)
lines=sc.textFile("/content/pagecounts-20160101-000000_parsed.out")
parsedlines=lines.map(parseLine).map(lambda title : (title, 1)).reduceByKey(lambda x , y: x+y)
final=parsedlines.max(lambda x:x[1])
print(final)
#list_Max=[]
#list_Max.insert(0,final)
#sc.parallelize(list_Max).saveAsTextFile('MaxTitle.tx')

('water', 118)


In [ ]:
#unique terms appearing in the page titles.
import re
def parse_data(line):
 colloms = line.split(" ")
 page_title=colloms[1]
 page_title2 = re.sub(r'[^a-zA-Z_]','', page_title).upper()
 return page_title2
def parse_pagetitle(line):
 colloms = line.split("_")
 return colloms
data =sc.textFile("/content/pagecounts-20160101-000000_parsed.out")
pageTitle_1=data.map(parse_data)
pageTitle_2=pageTitle_1.map(parse_pagetitle)
RDD=pageTitle_2.flatMap(lambda x:x).distinct().count()
list_count=[]
list_count.insert(0,RDD)

sc.parallelize(list_count).saveAsTextFile('UTerms1.tx')

In [ ]:
#maximum pageTitle
import re
def parse_data(line):
 colloms = line.split(" ")
 return (colloms[1])
data =sc.textFile("/content/pagecounts-20160101-000000_parsed.out")
RDD=data.map(parse_data)
pagetitle=RDD.map(lambda x: (x,1))

tt=pagetitle.groupByKey()
maxPageTitle=tt.mapValues(sum).map(lambda x:(x[1],x[0])).sortByKey(False)
maxPageTitle.take(10)
sc.parallelize(maxPageTitle.take(1)).saveAsTextFile('maxPageTitle11.tx')